In [1]:
cd /mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/notebooks/pipeline_integration

/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/notebooks/pipeline_integration


In [2]:
import os
import scanpy as sc
from os.path import join, exists
from os import listdir
import anndata
import scipy
import numpy as np
import sys

from utils import *

# convert counts into float32
# Convenience method for computing the size of objects
def print_size_in_MB(x):
    return '{:.3} MB'.format(x.__sizeof__()/1e6)

def print_size_in_MB_sparse_matrix(a):
    # a = scipy.sparse.csr_matrix(np.random.randint(10, size=(40, 3)))
    # x = a.data.nbytes + a.indptr.nbytes + a.indices.nbytes
    size = a.data.size/(1024**2)
    return '{:.3} MB'.format(size)

import warnings
warnings.filterwarnings("ignore")

### Prepare input for general run

In [3]:
scib_methods_path = 'scib_config.methods.yml'
lines = [r for r in open(scib_methods_path)]
methods_data = {}
curr_method = None
method_parms = []
for e in lines:
    if e.startswith('  ') and not e.startswith('   '):
        if curr_method is not None:
            methods_data[curr_method] = method_parms
        curr_method = e.strip()[:-1]
        method_parms = []
    else:
        method_parms.append(e.strip())
        
# last method
methods_data[curr_method] = method_parms

In [4]:
# methods_query =  ['harmony', 'combat', 'scgen', 'scanvi', 'scvi'] #  'scanorama', 'seurat', 'combat', 'bbknn'] # 'saucie'] # 'scgen']
methods_query =  ['scgen', 'scanvi', 'scvi'] #  'scanorama', 'seurat', 'combat', 'bbknn'] # 'saucie'] # 'scgen']

In [5]:
scib_template_path = 'scib_config.template.yml'
# exists(scib_template_path)
lines = ''.join([r for r in open(scib_template_path)])

import pandas as pd
from os import listdir
from os.path import abspath, exists, join
input_dir = '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input'
# batches = ['dataset', 'donor', 'batch.merged']
batches = ['batch_donor_dataset']
labels = ['cell.type']
# n_hvgs = [2000, 3000, 4000, 5000, 6000]

queries = []
for f in listdir(input_dir):
    if not f.endswith('.h5ad') or 'part' in f or 'ndatasets' in f:
        continue
    print(f)
    p = abspath(join(input_dir, f))
    
    name = 'retinal_' + f.split('_')[1] + '_' + f.split('_')[-1].replace('.h5ad', '')
    for batch in batches:
        for label in labels:
            queries.append([p, batch, label, name])
queries = pd.DataFrame(queries, columns=['file', 'batch_key', 'label_key', 'name',])

queries

input_500_cells_Chen+Hackney.h5ad
input_all_cells_Chen+Sanes.h5ad
input_500_cells_Chen.h5ad
input_all_cells_Chen.h5ad
input_all_cells_Chen+Scheetz.h5ad
input_500_cells_Chen+Wong.h5ad
input_500_cells_Chen+Sanes.h5ad
input_500_cells_Chen+Scheetz.h5ad
input_500_cells_Chen+Roska.h5ad
input_all_cells_all_fix5.h5ad
input_all_cells_all.h5ad
input_all_cells_Chen+Hackney+Roska.h5ad
input_all_cells_Chen+Wong.h5ad
input_500_cells_Chen+Hafler.h5ad
input_all_cells_Chen+Hafler.h5ad
input_all_cells_Chen+Roska.h5ad
input_all_cells_Chen+Hackney.h5ad
input_500_cells_all.h5ad


,file,batch_key,label_key,name
0,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Hackney
1,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Sanes
2,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen
3,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen
4,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Scheetz
5,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Wong
6,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Sanes
7,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Scheetz
8,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Roska
9,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_fix5


In [6]:
queries.shape

(18, 4)

In [7]:
import os
list(queries['file'])

['/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_500_cells_Chen+Hackney.h5ad',
 '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_all_cells_Chen+Sanes.h5ad',
 '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_500_cells_Chen.h5ad',
 '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_all_cells_Chen.h5ad',
 '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_all_cells_Chen+Scheetz.h5ad',
 '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_500_cells_Chen+Wong.h5ad',
 '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration

In [8]:
queries

,file,batch_key,label_key,name
0,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Hackney
1,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Sanes
2,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen
3,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen
4,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Scheetz
5,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Wong
6,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Sanes
7,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Scheetz
8,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Roska
9,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_fix5


In [9]:
# queries = queries[queries['file'].str.contains('Chen_fix5')]
queries

,file,batch_key,label_key,name
0,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Hackney
1,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Sanes
2,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen
3,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen
4,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Scheetz
5,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Wong
6,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Sanes
7,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Scheetz
8,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Roska
9,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_fix5


In [10]:
queries

,file,batch_key,label_key,name
0,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Hackney
1,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Sanes
2,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen
3,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen
4,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_Chen+Scheetz
5,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Wong
6,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Sanes
7,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Scheetz
8,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_500_Chen+Roska
9,/storage/groups/ml01/workspace/ignacio.ibarra/...,batch_donor_dataset,cell.type,retinal_all_fix5


In [11]:
# queries = queries[queries['name'].str.contains('Chen\+Hackney\+Roska')]

In [12]:

dataset_template = '''
  $query_name:
    batch_key: $batch_key
    label_key: $label_key
    assay: expression
    organism: human
    file: $path
'''
out_queries = ''
for ri, r in queries.iterrows():
    # print(ri, r.values)
    next_query = dataset_template.replace('$query_name', r['name'] + '_' + r['batch_key'] + '_' + r['label_key'])
    next_query = next_query.replace('$batch_key', r['batch_key']).replace('$label_key', r['label_key']).replace('$path', r['file'])
    out_queries += next_query

out_methods = ''
for q in methods_query:
    next_method = '  ' + q + ':\n    ' + '\n    '.join(methods_data[q])
    out_methods += next_method + '\n'
out = lines.replace('$METHODS_QUERY', out_methods)
out = out.replace('$DATA_SCENARIOS_QUERY', out_queries)

config_output_path = '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/scib/config_retinal_old.yaml'
writer = open(config_output_path, 'w')
writer.writelines(out + '\n')
writer.close()

overwrite = False
for n_sample_per_batch in [None]:
    # examine types, columns and others incorporated in the object
    
    code_n_cells = (('_' + str(n_sample_per_batch) if n_sample_per_batch is not None else ''))
    print(code_n_cells)
    print('# of cells (input argument)', n_sample_per_batch)
    code_output = (('_' + str(n_sample_per_batch) if n_sample_per_batch is not None else '_all'))
    output_path = '../../data/integration_march_2021/input/input%s_cells_all.h5ad' % code_output
    abspath_scib = os.path.abspath(output_path)        
    print(exists(abspath_scib), abspath_scib)
    
## write a single path with the DATA_scenarios
scenarios_path = config_output_path.replace('.yaml', '_scenarios.yaml')
writer = open(scenarios_path, 'w')
writer.writelines(out_queries + '\n')
writer.close()
# print(abspath(scenarios_path))


# of cells (input argument) None
True /mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/data/integration_march_2021/input/input_all_cells_all.h5ad


### Print commands that will be executed during the pipelin

In [13]:
dryrun_cmds_path = '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/scib/snakemake_dryrun_retinal.out'
dryrun_cmds_path_all = '/storage/groups/ml01/workspace/ignacio.ibarra/theislab/scib/snakemake_dryrun_retinal_all.out'

In [14]:
!readlink -f .

/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/notebooks/pipeline_integration


In [15]:
cd ~/workspace/ignacio.ibarra/theislab/scib

/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/scib


In [ ]:
!snakemake --configfile config_retinal.yaml --dryrun --printshellcmds --nolock  > $dryrun_cmds_path
!snakemake --configfile config_retinal.yaml --dryrun --printshellcmds --nolock -R integration_run_python  > $dryrun_cmds_path_all

In [ ]:
# !head $dryrun_cmds_path_all

In [ ]:
# cd '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/retinal_scRNAseq_integration/notebooks/pipeline_integration_2021_march'
queries = [q.strip() for q in open(dryrun_cmds_path) if q.strip().startswith('conda') and 'runIntegration' in q] #  in q and 'runIntegration'in q]
queries_postintegration = [q.strip() for q in open(dryrun_cmds_path_all) if q.strip().startswith('conda') and not 'runIntegration' in q] #  in q and 'runIntegration'in q]
queries_all = [q.strip() for q in open(dryrun_cmds_path_all) if q.strip().startswith('conda') and 'runIntegration' in q] #  in q and 'runIntegration'in q]


In [ ]:
len(queries), len(queries_all), len(queries_postintegration)

In [ ]:
for q in queries[:10]:
    print(q)
    print('')

### Generate sbatch query for GPU routines (e.g. scgen, scVI, scANVI)

In [ ]:
# queries_all

In [ ]:
q

In [ ]:
# to run GPU methods, we save the path of input queries into output
paths = []
for q in queries_all:
    s = q.split(' ')
    s = [si for si in s if not len(si.strip()) == 0]
    # print(s)

    args = ['--method', '-v', '-c', '-i', '-o', '--nepochs', '--nlayers', '--nhidden']
    v = []
    for a in args:
        found = False
        for si, x in enumerate(s):
            if x == a:
                found = True
                v.append(s[si + 1])
        if not found:
            v.append(None)
    # print(v)
    
    paths.append(v)
    


In [ ]:
cd ~/workspace/theislab/retinal_scRNAseq_integration/notebooks/pipeline_integration

In [ ]:
df = pd.DataFrame(paths, columns=['method', 'hvg', 'cell_type_key', 'input', 'output', 'nepochs', 'nlayers', 'nhidden'])

In [ ]:
# only 1K
df = df[df['input'].str.contains('1K')]

In [ ]:
list(df['output'])

In [ ]:
df['cell_type_key'] = np.where(df['method'].isin({'scvi', 'scgen'}),  'cell.type', df['cell_type_key'])
df
# for f in df['input']:
#     print(f)

In [ ]:
output_exists = []
input_exists = []
for ri, r in df.iterrows():
    output_exists.append(exists(r['output']))
    input_exists.append(exists(r['input']))
df['in.exists'] = input_exists
df['out.exists'] = output_exists

# df = df[~df['input'].str.contains('Roska')]
# df = df[~df['input'].str.contains('Hafler')]

df_all = df.copy()
df = df[~df['out.exists'] & df['in.exists']]
del df['in.exists']
del df['out.exists']


In [ ]:
df_all.shape

In [ ]:
df.shape

In [ ]:
# df['output']

In [ ]:
# list(df_all['output'])

In [ ]:
df['sample'] = df['output'].str.split('/').str[-5].astype(str).str.split('_').str[1]
df = df.sort_values('sample')
# df['output'].value_counts()
df = df[df['method'].str.split('_').str[0].isin({'scvi', 'scgen', 'scanvi'})]


In [ ]:
df.shape

In [ ]:
# list(df['output'])

In [ ]:
del df['sample']

In [ ]:
df.shape

In [ ]:
# df[((df['output'].str.contains('Wong')) & (df['method'].str.contains('scgen')))].reset_index(drop=True)
# df[((df['output'].str.contains('Wong')) & (df['method'].str.contains('scgen')))].reset_index(drop=True).to_csv('queries_gpu_methods.csv')

In [ ]:
df = df.reset_index(drop=True)
print(df.shape)
df.reset_index(drop=True).to_csv('queries_gpu_methods.csv')
# df
df.shape

In [ ]:
df

In [ ]:
df_all = df_all.reset_index(drop=True)
print(df_all.shape)
df_all.reset_index(drop=True).to_csv('queries_gpu_methods_all.csv')
# df
df_all.head(10)

In [ ]:
df_all

In [ ]:
df_all['out.exists'].value_counts()

In [ ]:
# list(df_all[~df_all['out.exists']]['out.exists'])

In [ ]:
for ri, r in df.iterrows():
    if not exists(r['output']):
        print(exists(r['input']), exists(r['output']), r['output'].replace('/storage/groups/ml01/workspace/ignacio.ibarra/', '~/workspace/theislab'))


In [ ]:
df.shape

In [ ]:
# this is the command one should run as a job array
print('sbatch --array=0-%i %s' % (df.shape[0] - 1, 'submit_gpu.sh'))

In [ ]:
!pwd

In [ ]:
df

### Generate a new config_retinal that will only contain the cases where output has been genera

In [ ]:
scib_wd_dir = '/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/theislab/scib'

In [ ]:
from os.path import join

In [ ]:
scib_methods_path = join(scib_wd_dir, 'config_retinal.yaml')
lines = [r for r in open(scib_methods_path)]
methods_data = {}
curr_method = None
method_parms = []
for e in lines:
    if e.startswith('  ') and not e.startswith('   '):
        if curr_method is not None:
            methods_data[curr_method] = method_parms
        curr_method = e.strip()[:-1]
        method_parms = []
    else:
        method_parms.append(e.strip())
        
# last method
methods_data[curr_method] = method_parms

### This indicates the amount of entries that have not finished

In [ ]:
entries = list(methods_data.keys())
for k in entries:
    if k.startswith('scvi') or k.startswith('scanvi') or k.startswith('scgen'):
        print(k, sum(~df_all[df_all['method'] == k]['out.exists']))
    else:
        del methods_data[k]


In [ ]:
methods_data

In [ ]:
for f in df['output']:
    print(exists(f), f)

## To calculate embeddings and other stuff, we only rely on cases where all output is available.

In [ ]:
done = {}
df_all['case'] = list(df_all['input'].str.split('/').str[-5])
for case, grp in df_all.groupby('method'):
    all_found = sum(grp['out.exists']) == grp.shape[0]
    print(case, grp.shape, sum(grp['out.exists']), all_found)

In [ ]:
len(queries_all)

In [ ]:
queries_postintegration = [q for q in queries_postintegration if 'save_embeddings.py' in q] #  in q or 'scripts/metrics.py' in q]
# print(len(queries_postintegration))
selected_cmds = []
for q in queries_postintegration:
    s = q.split(' ')
    input_pos = [pi for pi, p in enumerate(s) if p == '-i' or p == '--input'][0]
    input_path = s[input_pos + 1]
    # if exists(input_path):
    #     print(q)
    selected_cmds.append(q)
writer = open('queries_postintegration_%s.sh' % 'save_embeddings', 'w')
writer.writelines('\n'.join(selected_cmds) + '\n')
writer.close()

In [ ]:
len(selected_cmds)

In [ ]:
# this is the command one should run as a job array
print('sbatch --array=0-%i %s' % (len(selected_cmds) - 1, 'submit_cpu_save_embeddings.sh'))